# Non-Uniform Kochanek--Bartels Splines

Kochanek and Bartels (1984) mainly talk about uniform splines.
Only in section 4, "Adjustments for Parameter Step Size", they briefly mention the non-uniform case.

TODO: show equations for adjusted tangents

Unfortunately, this is wrong.

TODO: show why it is wrong.

Instead, we should start from the correct tangent vector for non-uniform Catmull--Rom splines:

\begin{equation}
\boldsymbol{\dot{x}}_i =
\frac{
(t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{
(t_{i+1} - t_i)(t_i - t_{i-1})(t_{i+1} - t_{i-1})
}
\end{equation}

## Parameters

In general
incoming tangent $\boldsymbol{\dot{x}}_i^{(-)}$ and
outgoing tangent $\boldsymbol{\dot{x}}_i^{(+)}$ at vertex $\boldsymbol{x}_i$:

\begin{align}
a_i &= (1 - T_i) (1 + C_i) (1 + B_i)\\
b_i &= (1 - T_i) (1 - C_i) (1 - B_i)\\
c_i &= (1 - T_i) (1 - C_i) (1 + B_i)\\
d_i &= (1 - T_i) (1 + C_i) (1 - B_i)
\end{align}

\begin{align}
\boldsymbol{\dot{x}}_i^{(+)} &= \frac{
a_i (t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
b_i (t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{(t_{i+1} - t_i) (t_i - t_{i-1}) (t_{i+1} - t_{i-1})}\\
\boldsymbol{\dot{x}}_i^{(-)} &= \frac{
c_i (t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
d_i (t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{(t_{i+1} - t_i) (t_i - t_{i-1}) (t_{i+1} - t_{i-1})}
\end{align}

In the calculation below, we consider
the outgoing tangent at $\boldsymbol{x}_0$ and
the incoming tangent at $\boldsymbol{x}_1$.

\begin{align}
a_0 &= (1 - T_0) (1 + C_0) (1 + B_0)\\
b_0 &= (1 - T_0) (1 - C_0) (1 - B_0)\\
c_1 &= (1 - T_1) (1 - C_1) (1 + B_1)\\
d_1 &= (1 - T_1) (1 + C_1) (1 - B_1)
\end{align}

\begin{align}
\boldsymbol{\dot{x}}_0^{(+)} &= \frac{
a_0 (t_1 - t_0)^2 (\boldsymbol{x}_0 - \boldsymbol{x}_{-1}) +
b_0 (t_0 - t_{-1})^2 (\boldsymbol{x}_1 - \boldsymbol{x}_0)
}{(t_1 - t_0) (t_0 - t_{-1}) (t_1 - t_{-1})}\\
\boldsymbol{\dot{x}}_1^{(-)} &= \frac{
c_1 (t_2 - t_1)^2 (\boldsymbol{x}_1 - \boldsymbol{x}_0) +
d_1 (t_1 - t_0)^2 (\boldsymbol{x}_2 - \boldsymbol{x}_1)
}{(t_2 - t_1) (t_1 - t_0) (t_2 - t_0)}
\end{align}

## Calculation

In [ ]:
%matplotlib inline
from IPython.display import display
import sympy as sp
sp.init_printing()

In [ ]:
from utility import NamedExpression, NamedMatrix

In [ ]:
x_1, x0, x1, x2 = sp.symbols('xbm_-1 xbm:3')

In [ ]:
t, t_1, t0, t1, t2 = sp.symbols('t t_-1 t:3')

Same control values as Catmull-Rom ...

In [ ]:
control_values_KB = sp.Matrix([x_1, x0, x1, x2])
control_values_KB

... but three additional parameters per vertex.
In our calculation, the parameters belonging to $\boldsymbol{x}_0$ and $\boldsymbol{x}_1$ are relevant:

In [ ]:
T0, T1 = sp.symbols('T:2')
C0, C1 = sp.symbols('C:2')
B0, B1 = sp.symbols('B:2')

In [ ]:
a = NamedExpression('a', (1 - T0) * (1 + C0) * (1 + B0))
b = NamedExpression('b', (1 - T0) * (1 - C0) * (1 - B0))
c = NamedExpression('c', (1 - T1) * (1 - C1) * (1 + B1))
d = NamedExpression('d', (1 - T1) * (1 + C1) * (1 - B1))
display(a, b, c, d)

In [ ]:
xd0 = NamedExpression(
    'xdotbm0',
    (a.name * (t1 - t0)**2 * (x0 - x_1) + b.name * (t0 - t_1)**2 * (x1 - x0)) /
    ((t1 - t0) * (t0 - t_1) * (t1 - t_1)))
xd1 = NamedExpression(
    'xdotbm1',
    (c.name * (t2 - t1)**2 * (x1 - x0) + d.name * (t1 - t0)**2 * (x2 - x1)) /
    ((t2 - t1) * (t1 - t0) * (t2 - t0)))
display(xd0, xd1)

In [ ]:
display(xd0.subs([a, b]))
display(xd1.subs([c, d]))

Same as with Catmull-Rom, try to find a transformation from cardinal control values to Hermite control values.
This can be used to get the full characteristic matrix.

In [ ]:
control_values_H = sp.Matrix([x0, x1, xd0.name, xd1.name])
control_values_H

From the [notebook about non-uniform Hermite splines](hermite-non-uniform.ipynb):

In [ ]:
M_H = NamedMatrix(
    r'{M_\text{H}}',
    sp.S('Matrix([[2, -2, Delta0, Delta0], [-3, 3, -2*Delta0, -Delta0], [0, 0, Delta0, 0], [1, 0, 0, 0]])'))
M_H

In [ ]:
M_KBtoH = NamedMatrix(r'{M_\text{KB$\to$H}}', 4, 4)
M_KB = NamedMatrix(r'{M_\text{KB}}', M_H.name * M_KBtoH.name)
M_KB

In [ ]:
sp.Eq(control_values_H, M_KBtoH.name * control_values_KB)

If we substitute the above definitions of $\boldsymbol{\dot{x}}_0$ and $\boldsymbol{\dot{x}}_1$, we can directly read off the matrix elements:

In [ ]:
M_KBtoH.expr = sp.Matrix([[expr.coeff(cv).simplify() for cv in control_values_KB]
                          for expr in control_values_H.subs([xd0.args, xd1.args]).expand()])
M_KBtoH

In [ ]:
deltas = [
    (t_1, -sp.Symbol('Delta_-1')),
    (t0, 0),
    (t1, sp.Symbol('Delta0')),
    (t2, sp.Symbol('Delta0') + sp.Symbol('Delta1'))
]

In [ ]:
M_KBtoH.subs(deltas)

In [ ]:
M_KB = M_KB.subs([M_H, M_KBtoH.subs(deltas)]).doit().expand()
M_KB

And for completeness' sake, its inverse:

In [ ]:
M_KB.I

TODO: plot some example curves